In [2]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException

In [3]:
driver = Driver(uc=True)   # undetected-chromedriver

In [4]:
BASE_URL = 'https://www.amazon.eg/s?k=%D8%AD%D9%88%D8%B6+%D8%A7%D8%B3%D8%AA%D8%A7%D9%86%D9%84%D8%B3&page=2&language=ar_AE&crid=12I8I0MDV10AV&qid=1767960872&sprefix=%D8%AD%D9%88%D8%B6+%D8%A7%D8%B3%D8%AA%D9%84%D8%B3%2Caps%2C140&xpid=-VKrmSH7HJ_EA&ref=sr_pg_2'

In [5]:
all_urls = []
driver.get(BASE_URL)
time.sleep(5)  # allow dynamic content to load


In [6]:
for i in range(1,8):
    links = driver.find_elements(By.CSS_SELECTOR, "a.a-link-normal.s-no-outline")
    for link in links:
        url = link.get_attribute("href")
        if url and "/dp/" in url:  # filter real product links
            all_urls.append(url)
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.s-pagination-next")
        if "s-pagination-disabled" in next_button.get_attribute("class"):
            break  # No more pages
        next_button.click()
        time.sleep(5)  # wait for next page to load
    except NoSuchElementException:
        break  # No "Next" button found, exit loop


In [7]:
all_urls

['https://www.amazon.eg/%D9%85%D8%B7%D8%A8%D8%AE-%D8%A7%D9%84%D8%B3%D8%AA%D8%A7%D9%86%D9%84%D8%B3-%D8%B3%D8%AA%D9%8A%D9%84-%D9%8A%D8%AB%D8%A8%D8%AA-%D8%A7%D9%84%D8%AD%D8%A7%D8%A6%D8%B7/dp/B0CXZS9JLS/ref=sr_1_53?crid=12I8I0MDV10AV&dib=eyJ2IjoiMSJ9.xWKNhoKYJcptHBd8zIjqKRJCk82K4-r6u68vrmGcx8Xs1CD_srebPwmMYPErOkeGWXeGGY7f76EK_O7moXqI6dQeJloxV6PMdAnYm3r1BclkmPHB5DB4HxNYPZCli7DnI7DdzXMnmeAHdA9j9mM-NYZBEIqBoHMeT5T_OTdtiDXmfiQkBYJVlsYB1brE0VRS_d6N4Iv5TwKCGrwPYAQUQxpoJ46KbdMHhJTuyO4niJ6uftdCVgakvAxtHR2qnwyAb9Q7bwAm5aMGXVS7ge1LKy-SayYJhtKGsPziFBC1kjw.52cdTu6e1vh-AcEz4TfUSIRdbfrB8hP6474tye9BsZI&dib_tag=se&keywords=%D8%AD%D9%88%D8%B6+%D8%A7%D8%B3%D8%AA%D8%A7%D9%86%D9%84%D8%B3&qid=1767963958&sprefix=%D8%AD%D9%88%D8%B6+%D8%A7%D8%B3%D8%AA%D9%84%D8%B3%2Caps%2C140&sr=8-53&xpid=-VKrmSH7HJ_EA',
 'https://www.amazon.eg/%D9%85%D8%B7%D8%A8%D8%AE-%D8%A7%D9%84%D8%B3%D8%AA%D8%A7%D9%86%D9%84%D8%B3-%D8%B3%D8%AA%D9%8A%D9%84-%D8%A7%D9%84%D8%AF%D8%A7%D8%AE%D9%84-%D8%A7%D9%84%D8%AE%D8%A7%D8%B1%D8%AC/dp/B0B2ZFNFJR/re

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def extract_amazon_reviews2(url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Referer": "https://www.google.com/"
    }

    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")
    # Product name
    product_tag = soup.find("span", {"id": "productTitle"})
    product_name = product_tag.get_text(strip=True) if product_tag else "Unknown Product"

    reviews = []

    review_blocks = soup.find_all("li", {"data-hook": "review"})

    for r in review_blocks:
        review_id = r.get("id")

        # Rating: numeric only
        rating_tag = r.find("i", {"data-hook": "review-star-rating"})
        if rating_tag:
            rating_text = rating_tag.text.strip()  # e.g., "5.0 out of 5 stars"
            rating_match = re.search(r"[\d.]+", rating_text)
            rating = float(rating_match.group()) if rating_match else None
        else:
            rating = None

        # Title: remove leading rating if present
        title_tag = r.find("a", {"data-hook": "review-title"})
        if title_tag:
            title_text = title_tag.get_text(strip=True)
            title = re.sub(r"^\d+(\.\d+)? out of 5 stars", "", title_text).strip()
        else:
            title = None

        # Date: remove "Reviewed in <Country> on"
        date_tag = r.find("span", {"data-hook": "review-date"})
        if date_tag:
            date_text = date_tag.text.strip()  # e.g., "Reviewed in Egypt on 26 July 2025"
            date_match = re.search(r"on (.+)$", date_text)
            date = date_match.group(1).strip() if date_match else date_text
        else:
            date = None

        # Body
        body_tag = r.find("span", {"data-hook": "review-body"})
        if body_tag:
            original = body_tag.find("span", class_="cr-original-review-content")
            body = original.text.strip() if original else body_tag.text.strip()
        else:
            body = None

        reviews.append({
            "product": product_name,
            "review_id": review_id,
            "rating": rating,
            "title": title,
            "date": date,
            "text": body
        })

    return reviews



In [ ]:
#testing
# url = 'https://www.amazon.eg/HP-%D9%84%D8%A7%D8%A8%D8%AA%D9%88%D8%A8-1920%C3%971080-1145G7-%D9%88%D9%8A%D9%86%D8%AF%D9%88%D8%B2/dp/B0FL2KJSKK/ref=sr_1_14?sr=8-14'
# a = extract_amazon_reviews2(url)
# print(a)

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-ae"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<!-- sp:end-feature:cs-optimization -->
<!-- sp:feature:csm:head-open-part2 -->
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=func

In [ ]:
import asyncio
import re
import os
import pandas as pd
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import nest_asyncio
from tqdm.asyncio import tqdm  # Specialized async version of tqdm

nest_asyncio.apply()

# 1. Hardware Limit: Use half of your cores
MAX_CONCURRENT_TASKS = max(1, os.cpu_count() // 1.3)
sem = asyncio.Semaphore(MAX_CONCURRENT_TASKS)

async def extract_amazon_reviews(url, pbar):
    # The semaphore ensures we don't exceed the CPU limit
    async with sem:
        async with async_playwright() as p:
            # Launch with low-resource flags
            browser = await p.chromium.launch(
                headless=True, 
                args=["--disable-gpu", "--no-sandbox", "--disable-dev-shm-usage"]
            )
            context = await browser.new_context(
                user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
            )
            page = await context.new_page()
            
            reviews = []
            try:
                await page.goto(url, wait_until="domcontentloaded", timeout=60000)
                # Wait for the specific review elements
                await page.wait_for_selector('li[data-hook="review"]', timeout=10000)
                
                html = await page.content()
                soup = BeautifulSoup(html, "html.parser")
                
                # --- Original Extraction Logic ---
                product_tag = soup.find("span", {"id": "productTitle"})
                product_name = product_tag.get_text(strip=True) if product_tag else "Unknown Product"
                
                review_blocks = soup.find_all("li", {"data-hook": "review"})
                for r in review_blocks:
                    rating_tag = r.find("i", {"data-hook": "review-star-rating"})
                    rating = float(re.search(r"[\d.]+", rating_tag.text).group()) if rating_tag else None

                    date_tag = r.find("span", {"data-hook": "review-date"})
                    date = re.search(r"on (.+)$", date_tag.text).group(1) if date_tag else None

                    body_tag = r.find("span", {"data-hook": "review-body"})
                    body = body_tag.get_text(strip=True) if body_tag else None

                    reviews.append({
                        "product": product_name,
                        "rating": rating,
                        "date": date,
                        "text": body
                    })
            except Exception:
                # If a page fails, we just return an empty list for that URL
                pass
            finally:
                await browser.close()
                pbar.update(1) # Move the progress bar forward
                return reviews

async def main(urls):
    # Initialize the progress bar
    pbar = tqdm(total=len(urls), desc="Scraping Amazon")
    
    # Create the tasks
    tasks = [extract_amazon_reviews(url, pbar) for url in urls]
    
    # Run them concurrently (subject to the semaphore limit)
    results = await asyncio.gather(*tasks)
    
    pbar.close()
    
    # Flatten results and save
    flat_list = [item for sublist in results for item in sublist]
    return flat_list


In [ ]:
# data = await main(all_urls)


Scraping Amazon: 100%|██████████| 258/258 [24:58<00:00,  5.81s/it]


In [ ]:
# df_all_reviews = pd.DataFrame(data)
# df_all_reviews.head()

""


In [28]:
all_reviews = []
for url in all_urls:
    reviews = extract_amazon_reviews2(url)
    all_reviews.extend(reviews)


In [29]:
df_all_reviews = pd.DataFrame(all_reviews)
df_all_reviews.head()

,product,review_id,rating,title,date,text
0,Smart Stainless Kitchen Sink Nano Ceramic Coat...,R35627U9VG5TAN,4.0,اشتريته ١١ الف وتم خفض السعر لنصف شي غريب ؟؟,12 May 2024,المنتج نفس الصورة وصل
1,Smart Stainless Kitchen Sink Nano Ceramic Coat...,RBLLTLQNGAAEB,2.0,مش احسن جودة,8 February 2025,وصلات ضعيفة جدا و الخامة مش احسن حاجة
2,Kroscro Black Stainless Steel Smart Kitchen Si...,R2CA3MVAPZUI1N,NaN,None,16 March 2025,Love my new sink it’s perfect and sturdy\nRead...
3,Kroscro Black Stainless Steel Smart Kitchen Si...,R3LPB38UUUZA5U,NaN,None,20 July 2025,this is the worst built sink i have ever seen....
4,Kroscro Black Stainless Steel Smart Kitchen Si...,R36GIQQWSYVAKT,NaN,None,6 September 2023,We had a professional plumber install it for u...


In [30]:
df_all_reviews.duplicated().sum()

np.int64(85)

In [32]:
df_all_reviews.drop_duplicates(inplace=True)

In [33]:
df_all_reviews.shape

(177, 6)

In [34]:
df_all_reviews.to_csv("sink_amazon_reviews.csv", index=False)